In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as f
import torch.utils.data as data
import torch.optim as optim
from torch.autograd import Variable
import torchvision
import numpy as np

In [3]:
from imageio import imread, imsave
import cv2
import os

in_dir='../dataset/color/'
img_paths = [x.path for x in os.scandir(in_dir) if x.name.endswith('.jpg') or x.name.endswith('.png')]
len(img_paths)

5578

In [4]:
photo = imread(img_paths[0])

In [5]:
import matplotlib.pyplot as plt
import PIL 

plt.figure("dog")
plt.imshow(photo)
plt.show()

<Figure size 640x480 with 1 Axes>

## 生成一个图片载入程序

In [6]:
imsize = 256
loader = torchvision.transforms.Compose([
#     transforms.Scale(imsize), 
    torchvision.transforms.ToTensor()
])

def image_loader(image_name):
    """load image, returns cuda tensor"""
    image = PIL.Image.open(image_name)
    image = loader(image).float()
    image = Variable(image, requires_grad=True)
    image = image.unsqueeze(0)  #this is for VGG, may not be needed for ResNet
    return image #assumes that you're using GPU

image = image_loader(img_paths[0])


In [7]:
image.shape

torch.Size([1, 3, 968, 1296])

## 定义网络

In [8]:
class BasicBlock(nn.Module):
    expansion = 1

    def __init__(self, inplanes, planes, stride=1, use_bias=True,downsample=None):
        super(BasicBlock, self).__init__()
        self.bn0=nn.BatchNorm2d(inplanes)
        self.conv1 = nn.Conv2d(inplanes, planes, kernel_size=5, stride=stride,padding=2, bias=use_bias)
        self.bn1 = nn.BatchNorm2d(planes)
        self.relu = nn.ReLU(inplace=True)
        self.conv2 = nn.Conv2d(planes, planes, kernel_size=5, stride=stride,padding=2, bias=use_bias)
        self.bn2 = nn.BatchNorm2d(planes)
        self.downsample = downsample
        self.stride = stride

    def forward(self, x):
        residual = x

        out = self.bn0(x)
        out = self.conv1(out)
        out = self.bn1(out)
        out = self.relu(out)
        out = self.conv2(out)
        out = self.bn2(out)
        if self.downsample is not None:
            residual = self.downsample(x)
        out += residual
        out = self.relu(out)
        return out


In [43]:
class DetectorModel(torch.nn.Module):

    def __init__(self, num_block=3, num_channels=16,conv_ksize=5,
                 use_bias=True, min_scale=2**-3, max_scale=1, num_scales=9):

        self.inplanes = num_channels
        self.num_blocks=num_block
        self.min_scale = min_scale
        self.max_scale=max_scale
        self.num_scales=num_scales

        super(DetectorModel, self).__init__()
        self.conv1 = nn.Conv2d(3, self.inplanes, kernel_size=conv_ksize, stride=1, padding=2,
                               bias=use_bias)
        self.bn1 = nn.BatchNorm2d(self.inplanes)
        self.layer=BasicBlock(self.inplanes, self.inplanes, stride=1, use_bias=True)
        self.soft_conv=nn.Conv2d(16, 1, kernel_size=conv_ksize, stride=1, padding=1,
                               bias=use_bias)

        self.ori_layer=nn.Conv2d(self.inplanes,2,kernel_size=conv_ksize, stride=1, padding=1,
                                bias=True )

    def forward(self, x):
        num_conv = 0

        x=self.conv1(x)
        num_conv+=1
        for i in range(self.num_blocks):
            x=self.layer(x)
            print(1)
        x=self.bn1(x)
        if self.num_scales == 1:
            scale_factors = [1.0]
        else:
            scale_log_factors = np.linspace(np.log(self.max_scale), np.log(self.min_scale), self.num_scales)
            scale_factors = np.exp(scale_log_factors)
        score_maps_list = []

        base_height_f = x.shape[2]
        base_width_f = x.shape[3]

        for i, s in enumerate(scale_factors):
            feat_height = (base_height_f * s + 0.5).astype(np.uint32)
            feat_width = (base_width_f * s + 0.5).astype(np.uint32)
            rs_feat_maps=torch.nn.functional.interpolate(x,[feat_height, feat_width])
            score_maps = self.soft_conv(rs_feat_maps)
            score_maps_list.append(score_maps)

        # orientation (initial map start from 0.0)
        # ori_W_init=torch.nn.init.zeros_()
        # ori_b_init = tf.constant(np.array([1, 0], dtype=np.float32))  # init with 1 for cos(q), 0 for sin(q)
        ori_b_init=torch.nn.init.constant(np.array([1,0], dtype=np.float32))

        self.ori_layer.bias.data.fill_(ori_b_init)
        ori_maps=self.ori_layer(x)
        ori_maps=f.normalize(ori_maps,dim=-1)

        endpoints={}
        endpoints['ori_maps'] = ori_maps
        endpoints['mso'] = True
        endpoints['multi_scores'] = True
        endpoints['scale_factors'] = scale_factors
        endpoints['pad_size'] = num_conv * (self.conv_ksize//2)
        return score_maps_list,endpoints
#         return x

In [44]:
model = DetectorModel()

In [45]:
optimizer = optim.SGD(model.parameters(),lr=0.001)

optimizer.zero_grad()
output = model(image)

1
1
1


/home/wang/.conda/envs/py3-theano/lib/python3.6/site-packages/ipykernel_launcher.py:52: UserWarning: nn.init.constant is now deprecated in favor of nn.init.constant_.


TypeError: constant_() missing 1 required positional argument: 'val'

In [12]:
output.shape

torch.Size([1, 16, 968, 1296])

# 可以看出来网络输入输出都是一样的尺寸

In [13]:
x=output
base_height_f = x.shape[2]
base_width_f = x.shape[3]

In [40]:
min_scale=1
max_scale=2**-3
num_scales=5

scale_log_factors = np.linspace(np.log(max_scale), np.log(min_scale), num_scales)
scale_factors = np.exp(scale_log_factors)

for i, s in enumerate(scale_factors):
    
    # scale are defined by extracted patch size (s of s*default_patch_size) so we need use inv-scale for resizing images
#     inv_s=1.0/s
    feat_height = (base_height_f * s + 0.5).astype(np.uint32)
    feat_width = (base_width_f * s + 0.5).astype(np.uint32)
    rs_feat_maps=torch.nn.functional.interpolate(x,[feat_height, feat_width])

In [42]:
rs_feat_maps.shape

torch.Size([1, 16, 968, 1296])

In [37]:
inv_s=1.0/scale_factors[0]

In [38]:
inv_s

7.999999999999998

In [31]:
feat_height

7744

In [21]:
feat_width

10368

In [35]:
scale_factors

array([0.125     , 0.2102241 , 0.35355339, 0.59460356, 1.        ])